# This notebook has the optimization codes; use fitting_na_16_plotter.ipynb to make plots

In [ ]:
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, stats
import bluepyopt as bpop
import curve_fitting as cf
import bluepyopt.deapext.algorithms as algo
import generalized_genSim_shorten_time as ggsd
import vclamp_evaluator_HMM as vcl_ev
import pickle
import time
from deap import tools
import multiprocessing
import eval_helper_na16 as eh16

In [ ]:
# just adjust these parameters, and then just run every block below to run the optimization

offspring_size = 1000
num_generations = 1000
output_log_file_name = 'jinan_na16_new_1.txt'
param_range_file = "./csv_files/param_stats_wide.csv"

In [ ]:
import numpy as np
import bluepyopt as bpop
import matplotlib.pyplot as plt

class Vclamp_evaluator(bpop.evaluators.Evaluator):

    def __init__(self, scaled):
        
        self.scaled = scaled
        
        # (val, min, max)
        param_range_dict = eh16.read_params_range(param_range_file)
        params_in_name = eh16.get_name_params_str()
        params_not_in_Range_dict = ['qq', 'tq']
        
        eh16.set_param(eh16.get_wt_params())
        
        # diff is mut - wild
        # first get baseline data points:
        gv_slope, v_half, top, bottom = cf.calc_act_obj("na16", is_HMM=False)
        self.act_v_half = v_half
        self.act_slope = gv_slope
        ssi_slope, v_half, top, bottom, tau0 = cf.calc_inact_obj("na16", is_HMM=False)
        self.inact_v_half = v_half
        self.inact_slope = ssi_slope
        self.tau0 = eh16.find_tau0()
        self.per_cur = eh16.find_persistent_current()
        
        def init_params():
            param_list = []
            print("here are the name, val, min, max of each parameter")
            for param in params_in_name:
                if param not in params_not_in_Range_dict:
                    print(param)
                    val = param_range_dict[param][0]
                    min_bound = param_range_dict[param][1]
                    max_bound = param_range_dict[param][2]
                    print(val)
                    print((min_bound, max_bound))
                    print("")
                    param_list.append(bpop.parameters.Parameter(param, value=val, bounds=(min_bound, max_bound)))
            return param_list

        print("init called")
        self.objectives = []
        self.objectives.append(bpop.objectives.Objective("V_half_Act"))
        self.objectives.append(bpop.objectives.Objective("V_half_inact"))
        self.objectives.append(bpop.objectives.Objective("slope_Act"))
        self.objectives.append(bpop.objectives.Objective("slope_inact"))
        self.objectives.append(bpop.objectives.Objective("tau0"))
        self.objectives.append(bpop.objectives.Objective("pers_curr"))
        self.params = init_params()
        
        goal_dict = eh16.read_mutant_protocols('./csv_files/mutant_protocols.csv', 'NA16_MUT')
        self.V_half_Act_diff_goal = goal_dict['dv_half_act']
        self.V_half_inact_diff_goal = goal_dict['dv_half_ssi']
        # slopes come in the 100 scale since it's a ratio, so we have to divide by 100
        self.slope_Act_ratio_goal = goal_dict['gv_slope']/100
        self.slope_inact_ratio_goal = goal_dict['ssi_slope']/100
        self.tau0_ratio_goal = goal_dict['tau0']/100
        self.per_cur_ratio_goal = goal_dict['persistent']/100
        
        
        
        print("\n\n\nhere are the goals:")
        print(self.V_half_Act_diff_goal)
        print(self.V_half_inact_diff_goal)
        print(self.slope_Act_ratio_goal)
        print(self.slope_inact_ratio_goal)
        print(self.tau0_ratio_goal)
        print(self.per_cur_ratio_goal)
        
        
    def evaluate_with_lists(self, param_values=[]):
        
        print("evaluate_with_lists is called")
        assert len(param_values) == len(self.params), 'no, they have to be equal...'
        
        currh = ggsd.Activation(channel_name = 'na16').h
        currh.sh_na16 = param_values[0]
        currh.tha_na16 = param_values[1]
        currh.qa_na16 = param_values[2]
        currh.Ra_na16 = param_values[3]
        currh.Rb_na16 = param_values[4]
        currh.thi1_na16 = param_values[5]
        currh.thi2_na16 = param_values[6]
        currh.qd_na16 = param_values[7]
        currh.qg_na16 = param_values[8]
        currh.mmin_na16 = param_values[9]
        currh.hmin_na16 = param_values[10]
        currh.q10_na16 = param_values[11]
        currh.Rg_na16 = param_values[12]
        currh.Rd_na16 = param_values[13]
        currh.thinf_na16 = param_values[14]
        currh.qinf_na16 = param_values[15]
        currh.vhalfs_na16 = param_values[16]
        currh.a0s_na16 = param_values[17]
        currh.zetas_na16 = param_values[18]
        currh.gms_na16 = param_values[19]
        currh.smax_na16 = param_values[20]
        currh.vvh_na16 = param_values[21]
        currh.vvs_na16 = param_values[22]
        currh.Ena_na16 = param_values[23]
        
        try:
            gv_slope, act_v_half, act_top, act_bottom = cf.calc_act_obj("na16", is_HMM=False)
            ssi_slope, inact_v_half, inact_top, inact_bottom, tau999 = cf.calc_inact_obj("na16", is_HMM=False)
            tau0 = eh16.find_tau0()
            per_cur = eh16.find_persistent_current()
        except:
            return [9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999]
        
        V_half_Act_diff = act_v_half - self.act_v_half
        V_half_inact_diff = inact_v_half - self.inact_v_half
        gv_slope_ratio = gv_slope/self.act_slope
        ssi_slope_ratio = ssi_slope/self.inact_slope
        tau0_ratio = tau0/self.tau0
        per_cur_ratio = per_cur/self.per_cur
        
        
        try:
            # eliminate outliers
            act = ggsd.Activation(channel_name = 'na16')
            act.genActivation()
            norm_act_y_val = sorted(list(act.gnorm_vec))
            act_fitted = eh16.get_fitted_act_conductance_arr(act.v_vec, gv_slope, act_v_half, act_top, act_bottom)

            inact = ggsd.Inactivation(channel_name = 'na16')
            inact.genInactivation()
            norm_inact_y_val = sorted(list(inact.inorm_vec))
            inac_fitted = eh16.get_fitted_inact_current_arr(inact.v_vec, ssi_slope, inact_v_half, inact_top, inact_bottom)
        except:
            return [9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999, 9999999999999999]
        
        
        if self.scaled:            
            return [(V_half_Act_diff/self.V_half_Act_diff_goal - 1)**2 * 1000,
                   (V_half_inact_diff/self.V_half_inact_diff_goal - 1)**2 * 1000,
                   (gv_slope_ratio/self.slope_Act_ratio_goal - 1)**2 * 1000,
                   (ssi_slope_ratio/self.slope_inact_ratio_goal - 1)**2 * 1000,
                   (tau0_ratio/self.tau0_ratio_goal - 1)**2 * 1000,
                   (per_cur_ratio/self.per_cur_ratio_goal - 1)**2 * 1000]
        else:
            return [(V_half_Act_diff - self.V_half_Act_diff_goal)**2,
                   (V_half_inact_diff - self.V_half_inact_diff_goal)**2,
                   (gv_slope_ratio - self.slope_Act_ratio_goal)**2,
                   (ssi_slope_ratio - self.slope_inact_ratio_goal)**2,
                   (tau0_ratio - self.tau0_ratio_goal)**2,
                   (per_cur_ratio - self.per_cur_ratio_goal)**2]

In [ ]:
# if scaled, then we will use the scaled scoring method that assigns equal weights to each parameter. Otherwise,
#      we will use natural weights
evaluator = Vclamp_evaluator(scaled = False)

In [ ]:
cur_log_file = output_log_file_name

gen_counter = 0
best_indvs = []
cp_freq = 1
old_update = algo._update_history_and_hof
def my_update(halloffame, history, population):
    global gen_counter,cp_freq
    if halloffame is not None:
        halloffame.update(population)
    
    if halloffame:
        best_indvs.append(halloffame[0])
        print(halloffame[0])
        f = open(cur_log_file, 'a')
        f.write(str(halloffame[0]) + '\n')
        f.close()
        #eh16.make_act_plots(halloffame[0])
        #eh16.make_inact_plots(halloffame[0])
    gen_counter = gen_counter+1
    print("Current generation: ", gen_counter)
    if gen_counter%cp_freq == 0:
        fn = '.pkl'
        save_logs(fn,best_indvs,population)

def my_record_stats(stats, logbook, gen, population, invalid_count):
    '''Update the statistics with the new population'''
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=invalid_count, **record)
    f = open(cur_log_file, 'a')
    f.write(str(logbook) + '\n\n\n')
    f.close()
    print('log: \n', logbook, '\n')
    output = open("log.pkl", 'wb')
    pickle.dump(logbook, output)
    output.close()

def save_logs(fn, best_indvs, hof):
    output = open("indv"+fn, 'wb')
    pickle.dump(best_indvs, output)
    output.close()
    output = open("hof"+fn, 'wb')
    pickle.dump(hof, output)


In [ ]:
#hof = tools.HallOfFame(1, similar=np.array_equal)
hof = tools.ParetoFront()
algo._update_history_and_hof = my_update
algo._record_stats = my_record_stats
pool = multiprocessing.Pool(processes=64)
deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=offspring_size, hof = hof, map_function=pool.map)
#, map_function=pool.map
#deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=5, hof = hof)
cp_file = './cp.pkl'

In [ ]:
start_time = time.time()
#pop, hof, log, hst = deap_opt.run(max_ngen=5, cp_filename=cp_file)
pop, hof, log, hst = deap_opt.run(max_ngen=num_generations, cp_filename=None)
end_time = time.time()
print(end_time - start_time)